In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
from elexmodel.handlers.data.LiveData import MockLiveDataHandler
from elexmodel.handlers.data.PreprocessedData import PreprocessedDataHandler
from elexmodel.handlers.data.CombinedData import CombinedDataHandler
from elexmodel.utils import math_utils
from elexmodel.handlers.config import ConfigHandler
from elexmodel.handlers import s3
from elexmodel.utils.file_utils import TARGET_BUCKET
from elexmodel.client import ModelClient

In [ ]:
import logging, sys
logging.disable(sys.maxsize)

In [ ]:
election_id = '2017-11-07_VA_G'
office = 'G'
geographic_unit_type = 'precinct'
estimands = ['turnout']
aggregates = ['postal_code', 'county_fips', 'unit']
features = ['ethnicity_likely_african_american', 'percent_bachelor_or_higher']
fixed_effects = []
robust = False
pi_method = "gaussian" #set for testing otherwise use `gaussian`
beta = 1
prediction_intervals = [0.9]
percent_reporting_threshold = 99

In [ ]:
config_handler = ConfigHandler(election_id, s3_client=s3.S3JsonUtil(TARGET_BUCKET))
estimand_baselines = config_handler.get_estimand_baselines(office, estimands)

In [ ]:
data_handler = MockLiveDataHandler(election_id, office, geographic_unit_type, estimands, s3_client=s3.S3CsvUtil(TARGET_BUCKET))
data_handler.shuffle(upweight={'county_classification': {'southwest': 30, 'southside': 15, 'central': 10}})

In [ ]:
preprocessed_data_handler = PreprocessedDataHandler(
    election_id, 
    office, 
    geographic_unit_type, 
    estimands, 
    estimand_baselines,
    s3_client=s3.S3CsvUtil(TARGET_BUCKET)
)
additional_data = preprocessed_data_handler.data
combined_data = CombinedDataHandler(additional_data, data_handler.data, estimands, geographic_unit_type)

In [ ]:
agg_functions = {'county_fips': 'first', 'county_classification': 'first'}
if 'district' in aggregates:
    agg_functions['district'] = 'first'
unit_to_county_info = (
    combined_data.data
    .groupby('geographic_unit_fips')
    .agg(agg_functions)
    .reset_index(drop=False)
)

In [ ]:
# For estimand = "turnout"
estimand = "turnout"

unit_mae = math_utils.compute_error(combined_data.data[f"results_{estimand}"], combined_data.data[f"last_election_results_{estimand}"], 'mae')
unit_mape = math_utils.compute_error(combined_data.data[f"results_{estimand}"], combined_data.data[f"last_election_results_{estimand}"], 'mape')
all_unit_maes = [unit_mae]
pred_unit_maes = [unit_mae]
all_unit_mapes = [unit_mape]
pred_unit_mapes = [unit_mape]
all_unit_within_pi = [np.nan]
pred_unit_within_pi = [np.nan]
all_unit_mean_length_pi = [np.nan]
pred_unit_mean_length_pi = [np.nan]

In [ ]:
assign_dict = {
    f'raw_results_{estimand}': lambda x: x[f'results_{estimand}'],
    f'last_election_results_{estimand}': lambda x: x[f'last_election_results_{estimand}']
}

In [ ]:

county_data = (
    combined_data.data
    .groupby('county_fips')
    .sum()
    .reset_index(drop=False)
    .assign(**assign_dict)
    [['county_fips', f'raw_results_{estimand}', f'last_election_results_{estimand}']]
)
county_mae = math_utils.compute_error(county_data[f"raw_results_{estimand}"].values, county_data[f"last_election_results_{estimand}"].values, 'mae')
county_mape = math_utils.compute_error(county_data[f"raw_results_{estimand}"].values, county_data[f"last_election_results_{estimand}"].values, 'mape')
county_maes = [county_mae]
county_mapes = [county_mape]
county_within_pi = [np.nan]
county_mean_length_pi = [np.nan]

In [ ]:
classification_data = (
    combined_data.data
    .groupby('county_classification')
    .sum()
    .reset_index(drop=False)
    .assign(**assign_dict)
    [['county_classification', f'raw_results_{estimand}', f'last_election_results_{estimand}']]
)

In [ ]:
if 'district' in aggregates:
    district_data = (
        combined_data.data
        .groupby(['postal_code', 'district'])
        .sum()
        .reset_index(drop=False)
        .assign(**assign_dict)
        [['postal_code', 'district', f'raw_results_{estimand}', f'last_election_results_{estimand}']]
    )
    district_mae = math_utils.compute_error(district_data[f'raw_results_{estimand}'].values, district_data[f'last_election_results_{estimand}'].values, 'mae')
    district_mape = math_utils.compute_error(district_data[f'raw_results_{estimand}'].values, district_data[f'last_election_results_{estimand}'].values, 'mape')
    district_maes = [district_mae]
    district_mapes = [district_mape]
    district_within_pi = [np.nan]
    district_mean_length_pi = [np.nan]

In [ ]:
state_data = (
    combined_data.data
    .groupby('postal_code')
    .sum()
    .reset_index(drop=False)
    .assign(**assign_dict)
    [['postal_code', f'raw_results_{estimand}', f'last_election_results_{estimand}']]
)
state_mae = math_utils.compute_error(state_data[f'raw_results_{estimand}'].values, state_data[f'last_election_results_{estimand}'].values, 'mae')
state_mape = math_utils.compute_error(state_data[f'raw_results_{estimand}'].values, state_data[f'last_election_results_{estimand}'].values, 'mape')
state_maes = [state_mae]
state_mapes = [state_mape]
state_within_pi = [np.nan]
state_mean_length_pi = [np.nan]

In [ ]:
state_metrics = []
county_metrics = []
district_metrics = []
classification_metrics = []

In [ ]:
n_reported = [0]
alpha_to_evaluate = 0.9
lower_string = f"lower_{alpha_to_evaluate}_{estimand}"
upper_string = f"upper_{alpha_to_evaluate}_{estimand}"
for i in range(20, data_handler.data.shape[0], 100):
    n_reported.append(i)
    current_reporting_data = data_handler.get_n_fully_reported(i)
    predictions = ModelClient().get_estimates(
        current_reporting_data,
        election_id, 
        office, 
        estimands, 
        prediction_intervals, 
        percent_reporting_threshold, 
        geographic_unit_type,
        features=features,
        aggregates=aggregates,
        fixed_effects=fixed_effects,
        pi_method=pi_method,
        beta=beta
    )
    
#     Turnout predictions

    unit_data_predictions = predictions['unit_data']
    all_units_to_evaluate = current_reporting_data.merge(
        unit_data_predictions, 
        how='left', 
        on=['postal_code', 'geographic_unit_fips']
    )
    
    # all units
    all_unit_maes.append(
        math_utils.compute_error(
            all_units_to_evaluate[f"raw_results_{estimand}"].values, 
            all_units_to_evaluate[f"pred_{estimand}"].values, 
            'mae'
        )
    )
    all_unit_mapes.append(
        math_utils.compute_error(
            all_units_to_evaluate[f"raw_results_{estimand}"].values, 
            all_units_to_evaluate[f"pred_{estimand}"].values, 
            'mape'
        )
    )
    all_unit_within_pi.append(
        math_utils.compute_frac_within_pi(
            all_units_to_evaluate[lower_string], 
            all_units_to_evaluate[upper_string], 
            all_units_to_evaluate[f"raw_results_{estimand}"]
        )
    )
    all_unit_mean_length_pi.append(
        math_utils.compute_mean_pi_length(
            all_units_to_evaluate[lower_string], 
            all_units_to_evaluate[upper_string], 
            all_units_to_evaluate[f"pred_{estimand}"]
        )
    )

    # predicted units only
    nonreporting_units_to_evaluate = data_handler.data_nonreporting.merge(
        unit_data_predictions, 
        how='left', 
        on=['postal_code', 'geographic_unit_fips']
    )
    
    pred_unit_maes.append(
        math_utils.compute_error(
            nonreporting_units_to_evaluate[f"raw_results_{estimand}"].values, 
            nonreporting_units_to_evaluate[f"pred_{estimand}"].values, 
            'mae'
        )
    )
    pred_unit_mapes.append(
        math_utils.compute_error(
            nonreporting_units_to_evaluate[f"raw_results_{estimand}"].values, 
            nonreporting_units_to_evaluate[f"pred_{estimand}"].values, 
            'mape'
        )
    )
    pred_unit_within_pi.append(
        math_utils.compute_frac_within_pi(
            nonreporting_units_to_evaluate[lower_string],
            nonreporting_units_to_evaluate[upper_string], 
            nonreporting_units_to_evaluate[f"raw_results_{estimand}"]
        )
    )
    pred_unit_mean_length_pi.append(
        math_utils.compute_mean_pi_length(
            nonreporting_units_to_evaluate[lower_string], 
            nonreporting_units_to_evaluate[upper_string],
            nonreporting_units_to_evaluate[f"pred_{estimand}"]
        )
    )
    
    # county
    if 'county_fips' in aggregates:
        county_data_predictions = predictions['county_data']
        counties_to_evaluate = county_data.merge(
            county_data_predictions, 
            on='county_fips'
        )
        county_maes.append(
            math_utils.compute_error(
                counties_to_evaluate[f"raw_results_{estimand}"].values, 
                counties_to_evaluate[f"pred_{estimand}"].values, 
                'mae'
            )
        )
        county_mapes.append(
            math_utils.compute_error(
                counties_to_evaluate[f"raw_results_{estimand}"].values, 
                counties_to_evaluate[f"pred_{estimand}"].values, 
                'mape'
            )
        )
        county_within_pi.append(
            math_utils.compute_frac_within_pi(
                counties_to_evaluate[lower_string],
                counties_to_evaluate[upper_string],
                counties_to_evaluate[f"raw_results_{estimand}"]
            )
        )
        county_mean_length_pi.append(
            math_utils.compute_mean_pi_length(
                counties_to_evaluate[lower_string],
                counties_to_evaluate[upper_string],
                counties_to_evaluate[f"pred_{estimand}"]
            )
        )
    
        unit_metrics_within_county = (
            nonreporting_units_to_evaluate
            .merge(unit_to_county_info, on='geographic_unit_fips')
            .groupby('county_fips')
            .apply(
                lambda x: pd.Series({
                    'within_pi': math_utils.compute_frac_within_pi(x[lower_string], x[upper_string],  x[f"raw_results_{estimand}"]),
                    'mape': math_utils.compute_error(x[f"raw_results_{estimand}"].values, x[f"pred_{estimand}"].values, 'mape')
                })
            )
        )
                    
        county_metrics_i = (
            counties_to_evaluate
            .merge(unit_metrics_within_county, on='county_fips')
            .assign(n_reported=i)
            .rename(columns={f'lower_{alpha_to_evaluate}_{estimand}': 'lower', f'upper_{alpha_to_evaluate}_{estimand}':'upper'})
            [['n_reported', 'county_fips', f"pred_{estimand}", f"raw_results_{estimand}", 'lower', 'upper', 'within_pi', 'mape']]
        )
        county_metrics.extend(county_metrics_i.values.tolist())
    
    # county classification
    if 'county_classification' in aggregates:
        classification_data_predictions = predictions['classification_data']
        classifications_to_evaluate = classification_data.merge(
            classification_data_predictions, 
            on='county_classification'
        )
        
        unit_metrics_within_classification = (
            nonreporting_units_to_evaluate
            .merge(unit_to_county_info, on='geographic_unit_fips')
            .groupby('county_classification')
            .apply(
                lambda x: pd.Series({
                    'within_pi': math_utils.compute_frac_within_pi(x[lower_string], x[upper_string], x[f"raw_results_{estimand}"]),
                    'mape': math_utils.compute_error(x[f"raw_results_{estimand}"].values, x[f"pred_{estimand}"].values, 'mape')
                })
            )
        )
               
        classification_metrics_i = (
            classifications_to_evaluate
            .merge(unit_metrics_within_classification, on='county_classification')
            .assign(n_reported=i)
            .rename(columns={f'lower_{alpha_to_evaluate}_{estimand}': 'lower', f'upper_{alpha_to_evaluate}_{estimand}':'upper'})
            [['n_reported', 'county_classification', f"pred_{estimand}", f"raw_results_{estimand}", 'lower', 'upper', 'within_pi']]
        )
        classification_metrics.extend(classification_metrics_i.values.tolist())
    
    # district
    if 'district' in aggregates:
        district_data_predictions = predictions['district_data']
        districts_to_evaluate = district_data.merge(
            district_data_predictions, 
            on=['postal_code', 'district']
        )
        district_maes.append(
            math_utils.compute_error(
                districts_to_evaluate[f"raw_results_{estimand}"].values, 
                districts_to_evaluate[f"pred_{estimand}"].values, 'mae'
            )
        )
        district_mapes.append(
            math_utils.compute_error(
                districts_to_evaluate[f"raw_results_{estimand}"].values, 
                districts_to_evaluate[f"pred_{estimand}"].values, 
                'mape'
            )
        )
        district_within_pi.append(
            math_utils.compute_frac_within_pi(
                districts_to_evaluate[lower_string],
                districts_to_evaluate[upper_string],
                districts_to_evaluate[f"raw_results_{estimand}"]
            )
        )
        district_mean_length_pi.append(
            math_utils.compute_mean_pi_length(
                districts_to_evaluate[lower_string],
                districts_to_evaluate[upper_string],
                districts_to_evaluate[f"raw_results_{estimand}"]
            )
        )
    
        unit_metrics_within_district = (
            nonreporting_units_to_evaluate
            .merge(unit_to_county_info, on='geographic_unit_fips')
            .groupby(['postal_code', 'district'])
            .apply(
                lambda x: pd.Series({
                    'within_pi': math_utils.compute_frac_within_pi(x[lower_string], x[upper_string], x[f"raw_results_{estimand}"]),
                    'mape': math_utils.compute_error(x[f"raw_results_{estimand}"].values, x[f"pred_{estimand}"].values, 'mape')
                })
            )
        )
    
        district_metrics_i = (
            districts_to_evaluate
            .merge(unit_metrics_within_district, on=['postal_code', 'district'])
            .assign(n_reported=i)
            .rename(columns={f'lower_{alpha_to_evaluate}_{estimand}': 'lower', f'upper_{alpha_to_evaluate}_{estimand}':'upper'})
            [['n_reported', 'postal_code', 'district', f"pred_{estimand}", f"raw_results_{estimand}", 'lower', 'upper', 'within_pi']]
        )
        district_metrics.extend(district_metrics_i.values.tolist())
    
    # state
    if 'postal_code' in aggregates:
        state_data_predictions = predictions['state_data']
        states_to_evaluate = state_data.merge(
            state_data_predictions, 
            on='postal_code'
        )
        state_maes.append(
            math_utils.compute_error(
                states_to_evaluate[f"raw_results_{estimand}"].values, 
                states_to_evaluate[f"pred_{estimand}"].values, 
                'mae'
            )
        )
        state_mapes.append(
            math_utils.compute_error(
                states_to_evaluate[f"raw_results_{estimand}"].values, 
                states_to_evaluate[f"pred_{estimand}"].values, 
                'mape'
            )
        )
        state_within_pi.append(
            math_utils.compute_frac_within_pi(
                states_to_evaluate[lower_string],
                states_to_evaluate[upper_string],
                states_to_evaluate[f"raw_results_{estimand}"]
            )
        )
        state_mean_length_pi.append(
            math_utils.compute_mean_pi_length(
                states_to_evaluate[lower_string],
                states_to_evaluate[upper_string],
                states_to_evaluate[f"pred_{estimand}"]
            )
        )
        
        unit_within_pi = (
            nonreporting_units_to_evaluate
            .groupby('postal_code')
            .apply(lambda x: math_utils.compute_frac_within_pi(x[lower_string], x[upper_string], x[f"raw_results_{estimand}"]))
            .reset_index(drop=False)
            .rename(columns={0: 'within_pi'})
        )
        states_metrics_i = (
            states_to_evaluate
            .assign(n_reported=i)
            .rename(columns={f'lower_{alpha_to_evaluate}_{estimand}': 'lower', f'upper_{alpha_to_evaluate}_{estimand}':'upper'})
            .merge(unit_within_pi, on='postal_code')
            [['n_reported', 'postal_code', f"pred_{estimand}", f"raw_results_{estimand}", 'lower', 'upper', 'within_pi']]
        )
        state_metrics.extend(states_metrics_i.values.tolist())

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(n_reported, all_unit_maes, label='all')
axs[0, 0].plot(n_reported, pred_unit_maes, label='pred')
axs[0, 0].title.set_text(f'unit MAE - {estimand}')
axs[0, 0].legend(loc="lower left")
axs[1, 0].plot(n_reported, all_unit_mapes, label='all')
axs[1, 0].plot(n_reported, pred_unit_mapes, label='pred')
axs[1, 0].title.set_text(f'unit MAPE - {estimand}')
axs[1, 0].legend(loc="lower left")
axs[0, 1].plot(n_reported, all_unit_within_pi, label='all')
axs[0, 1].plot(n_reported, pred_unit_within_pi, label='pred')
axs[0, 1].title.set_text(f'unit within PI - {estimand}')
axs[0, 1].set_ylim([0, 1.1])
axs[0, 1].axhline(y=alpha_to_evaluate, color='black')
axs[1, 0].legend(loc="lower left")
axs[1, 1].plot(n_reported, all_unit_mean_length_pi, label='all')
axs[1, 1].plot(n_reported, pred_unit_mean_length_pi, label='pred')
axs[1, 1].title.set_text(f'unit mean PI length - {estimand}')
axs[1, 1].legend(loc="lower left")

In [ ]:
if 'county_fips' in aggregates:
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs[0, 0].plot(n_reported, county_maes)
    axs[0, 0].title.set_text(f'county MAE - {estimand}')
    axs[1, 0].plot(n_reported, county_mapes)
    axs[1, 0].title.set_text(f'county MAPE - {estimand}')
    axs[0, 1].plot(n_reported, county_within_pi)
    axs[0, 1].title.set_text(f'county within PI - {estimand}')
    axs[0, 1].set_ylim([0, 1.1])
    axs[0, 1].axhline(y=alpha_to_evaluate, color='black')
    axs[1, 1].plot(n_reported, county_mean_length_pi)
    axs[1, 1].title.set_text(f'county mean PI length - {estimand}')

In [ ]:
if 'district' in aggregates:
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs[0, 0].plot(n_reported, district_maes)
    axs[0, 0].title.set_text(f'district MAE - {estimand}')
    axs[1, 0].plot(n_reported, district_mapes)
    axs[1, 0].title.set_text(f'district MAPE - {estimand}')
    axs[0, 1].plot(n_reported, district_within_pi)
    axs[0, 1].title.set_text(f'district within PI - {estimand}')
    axs[0, 1].set_ylim([0, 1.1])
    axs[0, 1].axhline(y=alpha_to_evaluate, color='black')
    axs[1, 1].plot(n_reported, district_mean_length_pi)
    axs[1, 1].title.set_text(f'district mean PI length - {estimand}')

In [ ]:
if 'postal_code' in aggregates:
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs[0, 0].plot(n_reported, state_maes)
    axs[0, 0].title.set_text(f'state MAE - {estimand}')
    axs[1, 0].plot(n_reported, state_mapes)
    axs[1, 0].title.set_text(f'state MAPE - {estimand}')
    axs[0, 1].plot(n_reported, state_within_pi)
    axs[0, 1].title.set_text(f'state within PI - {estimand}')
    axs[0, 1].set_ylim([0, 1.1])
    axs[0, 1].axhline(y=alpha_to_evaluate, color='black')
    axs[1, 1].plot(n_reported, state_mean_length_pi)
    axs[1, 1].title.set_text(f'state mean PI length - {estimand}')

In [ ]:
def plot_group(group, group_name, value_to_plot, position):
    global ax
    ax[position].plot(group.n_reported, group[value_to_plot], label=group[group_name].values[0])
    ax[position].legend(loc='best')
    if value_to_plot == 'within_pi':
        ax[position].set_ylim([0, 1.1])
        ax[position].axhline(y=alpha_to_evaluate, color='black')

    return group

In [ ]:
if 'county_fips' in aggregates:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    
    county_metrics_df = (
        pd.DataFrame(county_metrics, columns = ['n_reported', 'county_fips', f'pred_{estimand}', f'raw_results_{estimand}', 'lower', 'upper', 'within_pi', 'mape'])
        .query("mape > 0.3")
    )
    
    county_metrics_df.groupby('county_fips').apply(plot_group, 'county_fips', 'within_pi', 0)
    
    county_metrics_df.groupby('county_fips').apply(plot_group, 'county_fips', 'mape', 1)

In [ ]:
if 'county_classification' in aggregates:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    
    classification_metrics_df = (
        pd.DataFrame(classification_metrics, columns = ['n_reported', 'county_classification', f'pred_{estimand}', f'raw_results_{estimand}', 'lower', 'upper', 'within_pi'])
        .assign(error=lambda x: np.abs(x[f'raw_results_{estimand}'] - x[f'pred_{estimand}']) / x[f'raw_results_{estimand}'])
    )
    
    classification_metrics_df.groupby('county_classification').apply(plot_group, 'county_classification', 'within_pi', 0)
    
    classification_metrics_df.groupby('county_classification').apply(plot_group, 'county_classification', 'error', 1)

In [ ]:
if 'district' in aggregates:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    
    district_metrics_df = (
        pd.DataFrame(district_metrics, columns = ['n_reported', 'postal_code', f'pred_{estimand}', f'raw_results_{estimand}', 'lower', 'upper', 'within_pi'])
        .assign(error=lambda x: np.abs(x[f'raw_results_{estimand}'] - x[f'pred_{estimand}']) / x[f'raw_results_{estimand}'])
        .assign(state_district=lambda x: x.postal_code + '-' + x.district)
    )
    
    district_metrics_df.groupby(group_name).apply(plot_group, 'state_district', 'within_pi', 0)
    
    district_metrics_df.groupby(group_name).apply(plot_group, 'state_district', 'error', 1)

In [ ]:
if 'postal_code' in aggregates:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    
    state_metrics_df = (
        pd.DataFrame(state_metrics, columns = ['n_reported', 'postal_code', f'pred_{estimand}', f'raw_results_{estimand}', 'lower', 'upper', 'within_pi'])
        .assign(error=lambda x: np.abs(x[f"raw_results_{estimand}"] - x[f"pred_{estimand}"]) / x[f"raw_results_{estimand}"])
    )
    
    state_metrics_df.groupby('postal_code').apply(plot_group, 'postal_code', 'within_pi', 0)
    
    state_metrics_df.groupby('postal_code').apply(plot_group, 'postal_code', 'error', 1)

In [ ]:
state_metrics_df['within'] = (state_metrics_df.raw_results_turnout <= state_metrics_df.upper) & (state_metrics_df.raw_results_turnout >= state_metrics_df.lower)

In [ ]:
state_metrics_df